In [446]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

import urllib

print('Libraries imported.')

Libraries imported.


Get data from the web page.

In [447]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

toronto_html = BeautifulSoup(page, 'html.parser')

table_html = toronto_html.body.table.extract()

get table data from the web page

In [448]:
table = []
for x in table_html.find_all("td"):
    table.append(x.get_text())

table_zip = []
table_borough = []
table_neighborhood = []

for x in range(int(len(table)/3)):
    table_zip.append(table[x*3])

for x in range(int(len(table)/3)):
    table_borough.append(table[x*3+1])
    
for x in range(int(len(table)/3)):
    neighborhood = table[x*3+2]
    table_neighborhood.append(neighborhood[:-1])


create dataframe of the table

In [449]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
neighborhoods = pd.DataFrame(columns = column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


input table data to the dataframe

In [450]:
for x in range(int(len(table)/3)):
    neighborhoods = neighborhoods.append({'PostalCode': table_zip[x], 'Borough': table_borough[x], 'Neighborhood': table_neighborhood[x]}, ignore_index = True)
neighborhoods.head(20)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


clean the rows which have "Not assigned" attribute of Borough and reset the index

In [451]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop = True)
neighborhoods.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


change the "Not assigned" Neighborhood value to the value of Borough

In [452]:
for x in range(len(neighborhoods)):
    if neighborhoods.Neighborhood[x] == 'Not assigned':
        neighborhoods.Neighborhood[x] = neighborhoods.Borough[x]

neighborhoods.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


gourp the rows which have the same PostalCode

In [453]:
for x in range(len(neighborhoods)-1):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhoods.Neighborhood[x+1] = neighborhoods.Neighborhood[x] + ", " + neighborhoods.Neighborhood[x+1]

In [454]:
length = len(neighborhoods) - 1
neighborhood = neighborhoods
for x in range(length):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhood = neighborhood.drop(neighborhoods.index[x])
neighborhood = neighborhood.reset_index(drop = True)
neighborhood

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [455]:
neighborhood.shape

(103, 3)

get Latitude and Longitude data

In [456]:
geo = pd.read_csv(r"C:\Users\Rory\Downloads\Geospatial_Coordinates.csv")
geo.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


create Latitude and Longitude attributes

In [457]:
neighborhood['Latitude'] = 0.000000
neighborhood['Longitude'] = 0.000000
neighborhood.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",0.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",0.0,0.0
4,M7A,Queen's Park,Queen's Park,0.0,0.0
5,M9A,Etobicoke,Islington Avenue,0.0,0.0
6,M1B,Scarborough,"Rouge, Malvern",0.0,0.0
7,M3B,North York,Don Mills North,0.0,0.0
8,M4B,East York,"Woodbine Gardens, Parkview Hill",0.0,0.0
9,M5B,Downtown Toronto,"Ryerson, Garden District",0.0,0.0


merge the Latitude and Longitude data

In [458]:
for x in range(len(neighborhood)):
    for y in range(len(geo)):
        if neighborhood.PostalCode[x] == geo["Postal Code"][y]:
            neighborhood.Latitude[x] = geo.Latitude[y]
            neighborhood.Longitude[x] = geo.Longitude[y]

neighborhood.head(20)

C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
